**Устанавливаем все необходимые пакеты Python**

In [1]:
#Для расчета трассы полета
!pip install pyorbital
#Для получения TLE файла из базы
!pip install spacetrack

#Для работы с шейп-файлами
!pip install pyshp
!pip install shapely
!pip install shapefile

#Для работы с геоданными
!pip install geopandas
!pip install contextily
!pip install fiona

#Для создания интерактивной карты
!pip install folium

ERROR: Could not find a version that satisfies the requirement shapefile (from versions: none)
ERROR: No matching distribution found for shapefile


Импортируем библиотеки

In [2]:
#Библиотеки для построения трассировки КА
from datetime import datetime, date, timedelta
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
from pyorbital.orbital import Orbital

#Библиотеки для работы с шейпфайлами (shapefiles)
import shapefile
import fiona
from shapely.geometry import Point, Polygon, mapping

#Библиотеки для математических вычислений и моделирования
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import contextily as ctx
import pandas as pd
import csv
import math

#библиотека для всплывающего окна(ЮзерФрендли)
import tkinter as tk
from tkinter import filedialog, messagebox
import os

#Библиотека folium для создания интерактивной карты
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
print('All installed and imported!')

All installed and imported!


функция по созданию диалогового окна для взаимодействия с пользователем

In [3]:
def create_input_dialog():
    """Создает и возвращает диалоговое окно для ввода параметров"""
    root = tk.Tk()
    root.title("Ввод параметров")
    root.resizable(False, False)
    
    # Словарь для хранения виджетов и значений
    entries = {}
    values = {
        'papka': tk.StringVar(value=os.getcwd()),
        'USERNAME': tk.StringVar(),
        'PASSWORD': tk.StringVar(),
        'nomer': tk.StringVar(),
        'interval': tk.StringVar(value="30"),
        'vitkov': tk.StringVar(value="24"),
        'SK': tk.StringVar(value="3043"),
        'year': tk.StringVar(value="2025"),
        'mounth': tk.StringVar(value="1"),
        'day': tk.StringVar(value="1"),
        'max_ugol_obz': tk.StringVar(value="90"),
        'min_ugol_obz': tk.StringVar(value="-90"),
        'max_ugol_viz': tk.StringVar(value="50"),
        'min_ugol_viz': tk.StringVar(value="20")
    }
    
    # Функция для выбора папки
    def browse_folder():
        folder = filedialog.askdirectory()
        if folder:
            values['papka'].set(folder)
    
    # Функция проверки и сохранения данных
    def submit():
        errors = []
        # Проверка числовых полей
        numeric_fields = [
            'nomer', 'interval', 'vitkov', 'SK',
            'year', 'mounth', 'day',
            'max_ugol_obz', 'min_ugol_obz',
            'max_ugol_viz', 'min_ugol_viz'
        ]
        
        for field in numeric_fields:
            try:
                int(values[field].get())
            except ValueError:
                errors.append(f"Некорректное значение в поле '{field_labels[field]}'")
        
        if not values['papka'].get():
            errors.append("Путь к папке не указан")
        
        if errors:
            messagebox.showerror("Ошибки ввода", "\n".join(errors))
            return
        
        root.destroy()
    
    # Поля и их подписи
    field_labels = {
        'papka': "Рабочая папка:",
        'USERNAME': "Имя пользователя spacetrack.org:",
        'PASSWORD': "Пароль spacetrack.org:",
        'nomer': "NORAD ID КА:",
        'interval': "Временной интервал (секунды):",
        'vitkov': "Количество частей трассы:",
        'SK': "Система координат(метрическая):",
        'year': "Год:",
        'mounth': "Месяц:",
        'day': "День:",
        'max_ugol_obz': "Макс. широта наблюдения:",
        'min_ugol_obz': "Мин. широта наблюдения:",
        'max_ugol_viz': "Макс. угол визирования:",
        'min_ugol_viz': "Мин. угол визирования:"
    }
    
    # Создание элементов формы
    for i, field in enumerate(field_labels):
        # Подпись
        lbl = tk.Label(root, text=field_labels[field], anchor='e')
        lbl.grid(row=i, column=0, padx=5, pady=5, sticky='e')
        
        # Поле ввода
        if field == 'PASSWORD':
            entry = tk.Entry(root, textvariable=values[field], show="*")
        else:
            entry = tk.Entry(root, textvariable=values[field])
        
        entry.grid(row=i, column=1, padx=5, pady=5, sticky='we')
        entries[field] = entry
        
        # Кнопка выбора папки
        if field == 'papka':
            btn_browse = tk.Button(root, text="Обзор", command=browse_folder)
            btn_browse.grid(row=i, column=2, padx=5, pady=5)
    
    # Кнопки управления
    btn_frame = tk.Frame(root)
    btn_frame.grid(row=len(field_labels), column=0, columnspan=3, pady=10)
    
    btn_submit = tk.Button(btn_frame, text="OK", command=submit)
    btn_submit.pack(side=tk.LEFT, padx=10)
    
    btn_cancel = tk.Button(btn_frame, text="Отмена", command=root.destroy)
    btn_cancel.pack(side=tk.LEFT, padx=10)
    
    # Настройка размеров колонок
    root.columnconfigure(0, weight=0)
    root.columnconfigure(1, weight=1)
    root.columnconfigure(2, weight=0)
    
    root.mainloop()
    
    # Возвращаем результаты после закрытия окна
    if all(values[field].get() for field in values):
        return {k: v.get() for k, v in values.items()}
    return None

создание диалогового окна

In [4]:
if __name__ == "__main__":
    # Вызываем диалоговое окно
    input_data = create_input_dialog()
    papka = input_data['papka']
    USERNAME = input_data['USERNAME']
    PASSWORD = input_data['PASSWORD']
    nomer = int(input_data['nomer'])
    interval = int(input_data['interval'])
    vitkov = int(input_data['vitkov'])
    SK = int(input_data['SK'])
    year = int(input_data['year'])
    mounth = int(input_data['mounth'])
    day = int(input_data['day'])
    max_ugol_obz = int(input_data['max_ugol_obz'])
    min_ugol_obz = int(input_data['min_ugol_obz'])
    max_ugol_viz = int(input_data['max_ugol_viz'])
    min_ugol_viz = int(input_data['min_ugol_viz'])
    
    # Вычисляем шаг
    shag = int(24 / vitkov * 3600)

Функция получения начальных условий в формате TLE файла из базы данных spacetrack.org

In [5]:
def get_spacetrack_tle (sat_id, start_date, end_date, username, password, latest=False):
    st = SpaceTrackClient(identity=username, password=password) #идентификация пользователя
    if not latest:
        daterange = op.inclusive_range(start_date, end_date) #промежуток дат, на котором нужны TLE
        data = st.tle(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle', epoch = daterange) #начальные условия на заданном промежутке в прошлом
        print(daterange)
    else:
        data = st.tle_latest(norad_cat_id=sat_id, orderby='epoch desc', limit=1, format='tle') #актуальные начальные условия на сегодняшнюю дату
        print(date.today()) #дата актуального TLE

    if not data:
        return 0, 0

    f = open(papka + '/tle.txt', 'w')
    f.write(data[0:69] + '\n')
    f.write(data[70:139])
    f.close()
    return 'Txt create'

Функция расчёта подспутниковых точек на интервал времени в районе заданного дня

In [6]:
def create_orbital_track_shapefile_for_day (sat_id, track_day, start_time, end_time, step_seconds, output_shapefile, input_txt):
    # Для начала получаем TLE
    # Если запрошенная дата наступит в будущем, то запрашиваем самые последний набор TLE
    f = open(input_txt, 'r')
    tle_1 = f.readline()
    tle_2 = f.readline()

    # Создаём экземляр класса Orbital
    orb = Orbital("N", line1=tle_1, line2=tle_2)

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape = shapefile.Writer(output_shapefile)

    # Добавляем поля - идентификатор, время, широту и долготу
    # N - целочисленный тип, C - строка, F - вещественное число
    # Для времени придётся использовать строку, т.к. нет поддержки формата "дата и время"
    track_shape.field('ID','N', 40)
    track_shape.field('TIME','C', 40)
    track_shape.field('LAT','F', 40)
    track_shape.field('LON','F', 40)

    # Объявляем счётчики, i для идентификаторов, minutes для времени
    i = 0
    start_seconds = int((start_time - datetime(start_time.year, start_time.month, start_time.day, 0, 0, 0)).total_seconds())
    for seconds in range(start_seconds, start_seconds + int((end_time - start_time).total_seconds()), step_seconds):
        utc_hour = seconds // 3600
        utc_minutes = (seconds // 60) - (utc_hour * 60)
        utc_seconds = seconds - (utc_minutes * 60) - (utc_hour * 3600)

        # Сформируем строку для атрибута
        utc_string = str(utc_hour) + '-' + str(utc_minutes) + '-' + str(utc_seconds)
        # И переменную с временем текущего шага в формате datetime
        utc_time = datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds)

        # Считаем положение спутника
        lon, lat, alt = orb.get_lonlatalt(utc_time)

        # Создаём в шейп-файле новый объект
        # Определеяем геометрию
        track_shape.point(lon,lat)
        # и атрибуты
        track_shape.record(i, utc_string, lat, lon)

        # Не забываем про счётчики
        i += 1

    # Вне цикла нам осталось записать созданный шейп-файл на диск.
    # Т.к. мы знаем, что координаты положений ИСЗ были получены в WGS84
    # можно заодно создать файл .prj с нужным описанием

    try:
        prj = open("%s.prj" % output_shapefile.replace('.shp',''), "w")
        # Создаем переменную с описанием EPSG:4326 (WGS84)
        wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]'
        # Записываем её в файл .prj
        prj.write(wgs84_wkt)
        # И закрываем его
        prj.close()
        # Функцией save также сохраняем и сам шейп.
        track_shape.close()
        print('End of code!')
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

Функция расчёта подспутниковых точек на определённых широтах(зоны интересов) для сокращения вычислительной нагрузки

In [7]:
def create_orbital_track_shapefile_north (sat_id, track_day, start_time, end_time, step_seconds, output_shapefile, min_ugol, max_ugol, input_txt):
    # Для начала получаем TLE
    # Если запрошенная дата наступит в будущем, то запрашиваем самые последний набор TLE
    f = open(input_txt, 'r')
    tle_1 = f.readline()
    tle_2 = f.readline()

    # Создаём экземляр класса Orbital
    orb = Orbital("N", line1=tle_1, line2=tle_2)

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape = shapefile.Writer(output_shapefile)

    # Добавляем поля - идентификатор, время, широту и долготу
    # N - целочисленный тип, C - строка, F - вещественное число
    # Для времени придётся использовать строку, т.к. нет поддержки формата "дата и время"
    track_shape.field('ID','N', 40)
    track_shape.field('TIME','C', 40)
    track_shape.field('LAT','F', 40)
    track_shape.field('LON','F', 40)

    # Объявляем счётчики, i для идентификаторов, minutes для времени
    i = 0
    start_seconds = int((start_time - datetime(start_time.year, start_time.month, start_time.day, 0, 0, 0)).total_seconds())
    for seconds in range(start_seconds, start_seconds + int((end_time - start_time).total_seconds()), step_seconds):
        utc_hour = seconds // 3600
        utc_minutes = (seconds // 60) - (utc_hour * 60)
        utc_seconds = seconds - (utc_minutes * 60) - (utc_hour * 3600)

        # Сформируем строку для атрибута
        utc_string = str(utc_hour) + '-' + str(utc_minutes) + '-' + str(utc_seconds)
        # И переменную с временем текущего шага в формате datetime
        utc_time = datetime(track_day.year, track_day.month, track_day.day, utc_hour, utc_minutes, utc_seconds)

        # Считаем положение спутника
        lon, lat, alt = orb.get_lonlatalt(utc_time)
        if lat>=min_ugol and lat<=max_ugol:
            # Создаём в шейп-файле новый объект
            # Определеяем геометрию
            track_shape.point(lon,lat)
            # и атрибуты
            track_shape.record(i, utc_string, lat, lon)

        # Не забываем про счётчики
        i += 1

    # Вне цикла нам осталось записать созданный шейп-файл на диск.
    # Т.к. мы знаем, что координаты положений ИСЗ были получены в WGS84
    # можно заодно создать файл .prj с нужным описанием

    try:
        prj = open("%s.prj" % output_shapefile.replace('.shp',''), "w")
        # Создаем переменную с описанием EPSG:4326 (WGS84)
        wgs84_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]'
        # Записываем её в файл .prj
        prj.write(wgs84_wkt)
        # И закрываем его
        prj.close()
        # Функцией save также сохраняем и сам шейп.
        track_shape.close()
        print('End of code!')
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

**Функции считывания координат из геодатасета GeoData:**

Функция определения координат группы полигонов

In [8]:
def polygon_to_coords(buff):
  buff_coords = []
  pol = []
  for i in range(len(buff)):
    poly_mapped = mapping(buff['geometry'][i])
    poly_coordinates = poly_mapped['coordinates'][0]
    pol.append([])
    for j in range(len(poly_coordinates)):
      pol[i].append([list(poly_coordinates[j])[0], list(poly_coordinates[j])[1]]) #Возможно нужно поменять 0 и 1 местами!!!

    buff_coords.append(pol[i])

  return buff_coords

Функция определения координат группы точек

In [9]:
def point_to_coords(vitok_1):
  point_coords = []
  for i in range(len(list(vitok_1['geometry']))):
    point_coords.append([mapping(list(vitok_1['geometry'])[i])['coordinates'][1], mapping(list(vitok_1['geometry'])[i])['coordinates'][0]])
  return point_coords

Функция определения координат отдельного одного полигона

In [10]:
def poly_to_coords(left_bord_2g):
  coords = []
  for i in range(len(list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0]))):
    coords.append([list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0][i])[1], list(mapping(left_bord_2g['geometry'])['features'][0]['geometry']['coordinates'][0][i])[0]])
  return coords

**Функция получения данных о проекции Земли по EPSG**

In [11]:
def getWKT_PRJ(epsg_code):

    ## import required
    import urllib.request

    ## access projection information
    response = urllib.request.urlopen("http://spatialreference.org/ref/epsg/{0}/prettywkt/".format(epsg_code))

    ## decode the response from bytes to string
    wkt = response.read().decode('utf-8')

    ## return with no linebreaks or spaces
    return wkt.replace("\n", "").replace(" ", "")

**Функция получения точек съёмки по заданному углу визирования**

In [12]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Библиотека pyshp понадобится для создания шейп-файла


#Просчет трассы с учётом углов визирования
def ugol_visir(input_shapefile, output_shapefile_left, output_shapefile_right, ugol):
    #Вычисление расстояния от надира в км
    import math
    import numpy as np

    R = 6371302 #м
    h = 518833 #м

    gamma = math.degrees(math.asin((h+R)*math.sin(math.radians(ugol))/R)) - ugol
    L = 2*math.pi*R*gamma/360 #Перпендикуляное расстояние от точки надира в м

    lat = []
    long = []
    #Из шейп файла считываем координаты точек в надире
    nadir = gpd.read_file(input_shapefile)
    nadir = nadir.to_crs(epsg=SK) #перевести подспутниковые точки в проекцию EPSG3043
    coords_nadir = point_to_coords(nadir)
    for i in range(len(coords_nadir)):
      if coords_nadir[i][1]!= math.inf:
        lat.append(coords_nadir[i][1])
        long.append(coords_nadir[i][0])

    # Создаём экземпляр класса Writer для создания шейп-файла, указываем тип геометрии
    track_shape1 = shapefile.Writer(output_shapefile_left)
    track_shape2 = shapefile.Writer(output_shapefile_right)

    # Добавляем поля - широту и долготу
    track_shape1.field('LAT','F',40)
    track_shape1.field('LON','F',40)
    track_shape2.field('LAT','F',40)
    track_shape2.field('LON','F',40)

    lat_new_r = []
    lat_new_l = []
    long_new_r = []
    long_new_l = []

    for k in range(len(lat)-1):
      if lat[k+1]-lat[k] ==0:
          #Восхождение
          if (long[k]<long[k+2]):
              #Правый борт
              lat_new_r.append(lat[k]+L)
              long_new_r.append(long[k])
              #Левый борт
              lat_new_l.append(lat[k]-L)
              long_new_l.append(long[k])

          #Нисхождение
          if (long[k]>long[k+2]):
              #Правый борт
              lat_new_r.append(lat[k]+L)
              long_new_r.append(long[k])
              #Левый борт
              lat_new_l.append(lat[k]-L)
              long_new_l.append(long[k])
      elif long[k+1]-long[k] ==0:
        #Восхождение
          if (lat[k]<lat[k+2]):
              #Правый борт
              lat_new_r.append(lat[k])
              long_new_r.append(long[k]+L)
              #Левый борт
              lat_new_l.append(lat[k])
              long_new_l.append(long[k]-L)

          #Нисхождение
          if (lat[k]>lat[k+2]):
              #Правый борт
              lat_new_r.append(lat[k])
              long_new_r.append(long[k]-L)
              #Левый борт
              lat_new_l.append(lat[k])
              long_new_l.append(long[k]+L)
      else:
          delt_s = abs(lat[k+1]-lat[k])
          delt_d = abs(long[k+1]-long[k])
          #Угол отклонения скорости
          beta = math.atan(delt_s/delt_d) #!

          #Восхождение
          if (lat[k]<lat[k+1]):
            #Правый борт
            lat_new_r.append(lat[k]+L*math.cos(beta))
            long_new_r.append(long[k]+L*math.sin(beta))
            #Левый борт
            lat_new_l.append(lat[k]-L*math.cos(beta))
            long_new_l.append(long[k]-L*math.sin(beta))

          #Нисхождение
          if (lat[k]>lat[k+1]):
            #Правый борт
            lat_new_r.append(lat[k]+L*math.cos(beta))
            long_new_r.append(long[k]-L*math.sin(beta))
            #Левый борт
            lat_new_l.append(lat[k]-L*math.cos(beta))
            long_new_l.append(long[k]+L*math.sin(beta))


    for i in range(len(long_new_r)):

        # Создаём в шейп-файле новый объект
        # Определеяем геометрию
        track_shape1.point(lat_new_r[i],long_new_r[i])
        # и атрибуты
        track_shape1.record(lat_new_r[i],long_new_r[i])

        track_shape2.point(lat_new_l[i],long_new_l[i])
        track_shape2.record(lat_new_l[i],long_new_l[i])

    try:
        # Создаем файл .prj с тем же именем, что и выходной .shp
        prj = open("%s.prj" % output_shapefile_left.replace('.shp',''), "w")
        ## call the function and supply the epsg code
        epsg_info = getWKT_PRJ(SK)
        prj.write(epsg_info)
        # И закрываем его
        prj.close()
        # Создаем файл .prj с тем же именем, что и выходной .shp
        prj = open("%s.prj" % output_shapefile_right.replace('.shp',''), "w")
         ## call the function and supply the epsg code
        epsg_info = getWKT_PRJ(SK)
        prj.write(epsg_info)
        # И закрываем его
        prj.close()


        # Функцией save также сохраняем и сам шейп.
        #track_shape.save(output_shapefile)
        track_shape1.close()
        track_shape2.close()
    except:
        # Вдруг нет прав на запись или вроде того...
        print ('Unable to save shapefile')
        return

    print('Ready')


**Функция получения полигонов полос визирования для отдельного витка**

In [13]:
def add_vitok(input_shapefile, min_ugol, max_ugol, output_l1, output_l2, output_r1, output_r2):
  ugol_visir(input_shapefile,'min_left.shp','min_right.shp', min_ugol)
  ugol_visir(input_shapefile,'max_left.shp','max_right.shp', max_ugol)
  import warnings
  warnings.filterwarnings('ignore')

  u20_r = gpd.read_file('min_right.shp')
  u55_r = gpd.read_file('max_right.shp')
  u20_l = gpd.read_file('min_left.shp')
  u55_l = gpd.read_file('max_left.shp')

  u20_r = u20_r.to_crs(epsg=4326)
  u55_r = u55_r.to_crs(epsg=4326)
  u20_l = u20_l.to_crs(epsg=4326)
  u55_l = u55_l.to_crs(epsg=4326)

  print('Try to do polygons')
  min_l_lat = []
  min_l_lon = []
  max_l_lat = []
  max_l_lon = []

  min_r_lat = []
  min_r_lon = []
  max_r_lat = []
  max_r_lon = []
  for i in range(len(u20_r)):
    min_r_lat.append(point_to_coords(u20_r)[i][0])
    min_r_lon.append(point_to_coords(u20_r)[i][1])

  for i in range(len(u20_l)):
    min_l_lat.append(point_to_coords(u20_l)[i][0])
    min_l_lon.append(point_to_coords(u20_l)[i][1])

  for i in range(len(u55_r)):
    max_r_lat.append(point_to_coords(u55_r)[i][0])
    max_r_lon.append(point_to_coords(u55_r)[i][1])

  for i in range(len(u55_l)):
    max_l_lat.append(point_to_coords(u55_l)[i][0])
    max_l_lon.append(point_to_coords(u55_l)[i][1])

  print('Modeling')

  #Учет смены полушарий - переход через 180 меридиан
  global ind20
  ind20 = 0


  for i in range(len(u20_l)-1):
    if min_l_lon[i]>0 and min_l_lon[i-1]<0:
      ind20 = i
      break

  if (ind20==0):
    schema = {
        'geometry':'Polygon'
    }

    left_bord = fiona.open(output_l1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    for k in range(len(min_l_lon)):
      if min_l_lon[k]>=-150:
        xyList.append((min_l_lon[k], min_l_lat[k]))

    for k in range(len(max_l_lon)-1, -1, -1):
      if min_l_lon[k]>=-150:
        xyList.append((max_l_lon[k], max_l_lat[k]))

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord.write(rowDict)
    left_bord.close()
    print('Left has done')

    right_bord = fiona.open(output_r1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    for k in range(len(min_r_lon)):
      if min_r_lon[k]>=-150:
        xyList.append((min_r_lon[k], min_r_lat[k]))

    for k in range(len(max_r_lon)-1, -1, -1):
      if max_r_lon[k]>=-150:
        xyList.append((max_r_lon[k], max_r_lat[k]))
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord.write(rowDict)
    right_bord.close()
    print('Right has done')

  else:
    schema = {
        'geometry':'Polygon'
    }

    left_bord_1 = fiona.open(output_l1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=0
    while  k!=ind20-1:#!-1 изначально
      if min_l_lon[k]<=160:
        xyList.append((min_l_lon[k], min_l_lat[k]))
      k+=1

    k=ind20-2 #!
    while  k!=0-1:
      if max_l_lon[k]<=160:
        xyList.append((max_l_lon[k], max_l_lat[k]))
      k-=1

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord_1.write(rowDict)
    left_bord_1.close()
    print('Left 1 has done')

    right_bord_1 = fiona.open(output_r1, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=0
    while  k!=ind20: #!!!
      if min_r_lon[k]<=160:
        xyList.append((min_r_lon[k], min_r_lat[k]))
      k+=1

    k=ind20 #!
    while  k!=0: #!
      if max_r_lon[k]<=160:
        xyList.append((max_r_lon[k], max_r_lat[k]))
      k-=1
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord_1.write(rowDict)
    right_bord_1.close()
    print('Right 1 has done')

    left_bord_2 = fiona.open(output_l2, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=ind20
    while  k!=len(u20_l)-1: #-1 изначально
      if min_l_lon[k]>=-160:
        xyList.append((min_l_lon[k], min_l_lat[k]))
      k+=1

    k=len(u20_l)-1
    while  k!=ind20: #!
      if max_l_lon[k]>=-160:
        xyList.append((max_l_lon[k], max_l_lat[k]))
      k-=1

    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    left_bord_2.write(rowDict)
    left_bord_2.close()
    print('Left 2 has done')

    right_bord_2 = fiona.open(output_r2, mode='w', driver='ESRI Shapefile',
              schema = schema, crs = "EPSG:4326")
    xyList = []

    k=ind20+1 #!!
    while  k!=len(min_r_lon)-1:
      if min_r_lon[k]>=-160:
        xyList.append((min_r_lon[k], min_r_lat[k]))
      k+=1

    k=len(min_r_lon)-1 #-1 изначально
    while  k!=ind20+1: #!+1 изначально
      if max_r_lon[k]>=-160:
        xyList.append((max_r_lon[k], max_r_lat[k]))
      k-=1
    rowDict = {
    'geometry' : {'type':'Polygon',
                    'coordinates': [xyList]}
    }
    right_bord_2.write(rowDict)
    right_bord_2.close()
    print('Right 2 has done')

**Блок расчёта и создания полигонов полос обзора.**

In [14]:
get_spacetrack_tle (nomer, datetime(year, mounth, day), datetime(year, mounth, day+1), USERNAME, PASSWORD, latest=True)

2025-07-18


'Txt create'

In [15]:
b_d = datetime(year, mounth, day, 0, 0, 0)
b= b_d.hour+b_d.minute+b_d.second
i=vitkov
while i!=0:
    c= b+shag
    print(datetime(year, mounth, day, b//3600, (b%3600)//60, b%60))
    if c//3600 ==24:
        c=0
        #day+=1
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_north(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, 23, 59, 59), interval, papka+'/{}/vitok_{}_cut_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), min_ugol_obz, max_ugol_obz, papka + '/tle.txt')
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, 23, 59, 59), interval, papka+'/{}/vitok_{}_pol_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    else:
        print(datetime(year, mounth, day, c//3600, (c%3600)//60, c%60))
        create_orbital_track_shapefile_north(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, c//3600, (c%3600)//60, c%60), interval, papka+'/{}/vitok_{}_cut_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), min_ugol_obz, max_ugol_obz, papka + '/tle.txt')
        create_orbital_track_shapefile_for_day(nomer, date(year, mounth, day), datetime(year, mounth, day, b//3600, (b%3600)//60, b%60), datetime(year, mounth, day, c//3600, (c%3600)//60, c%60), interval, papka+'/{}/vitok_{}_pol_{}.shp'.format(vitkov-i+1, vitkov-i+1, nomer), papka + '/tle.txt')
    b=c
    i-=1
for i in range(1, vitkov+1, 1):
    add_vitok(papka+'/{}/vitok_{}_cut_{}.shp'.format(i, i, nomer), min_ugol_viz, max_ugol_viz, papka+"/{}/vitok_{}_cut_{}_l1_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_l2_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_r1_test.shp".format(i, i, nomer), papka+"/{}/vitok_{}_cut_{}_r2_test.shp".format(i, i, nomer))
print('Подспутниковые точки и зоны визирования сохранены в указанную папку: ' + papka)

2025-07-17 00:00:00
2025-07-17 01:00:00
End of code!
End of code!
2025-07-17 01:00:00
2025-07-17 02:00:00
End of code!
End of code!
2025-07-17 02:00:00
2025-07-17 03:00:00
End of code!
End of code!
2025-07-17 03:00:00
2025-07-17 04:00:00
End of code!
End of code!
2025-07-17 04:00:00
2025-07-17 05:00:00
End of code!
End of code!
2025-07-17 05:00:00
2025-07-17 06:00:00
End of code!
End of code!
2025-07-17 06:00:00
2025-07-17 07:00:00
End of code!
End of code!
2025-07-17 07:00:00
2025-07-17 08:00:00
End of code!
End of code!
2025-07-17 08:00:00
2025-07-17 09:00:00
End of code!
End of code!
2025-07-17 09:00:00
2025-07-17 10:00:00
End of code!
End of code!
2025-07-17 10:00:00
2025-07-17 11:00:00
End of code!
End of code!
2025-07-17 11:00:00
2025-07-17 12:00:00
End of code!
End of code!
2025-07-17 12:00:00
2025-07-17 13:00:00
End of code!
End of code!
2025-07-17 13:00:00
2025-07-17 14:00:00
End of code!
End of code!
2025-07-17 14:00:00
2025-07-17 15:00:00
End of code!
End of code!
2025-07-17